<h1 style="color:black;text-align:center">Capstone Project</h1>
<h1 style="color:slateblue;text-align:center">IBM Professional Certificate in Data Science</h1><br><br>
<h2 style="color:gray;text-align:center"><i>Opening a African Restaurant in Downtown Toronto, Canada</i></h2><br>
<img align="center"  src="https://www.senecacollege.ca/content/dam/projects/seneca/homepage-assets/homepage_intl.jpg" width=700 height=400></img><br></br>
<p style="color:slateblue;text-align:center;font-size:24px">By Isaac Nyamekye<br><br><i>April 2020</i></p>

---
# Table of contents
* [Background and Business Problem](#introduction)
* [Data](#data)

# Background and Business Problem <a name="introduction"></a>

<p style="font-size:24px">Toronto is the most populous city in Canada and the provincial capital of Ontario. The city is very diverse and home to a large immigrant population. Because of the cultural diversity of its inhabitants, the city has a dynamic and diverse culinary scene.<br><br>
However, there are a few Ghanaian (West Africa) restaurants, especially, in downtown Toronto. An African restaurant chain would like to change this by opening a couple of Ghanaian restaurants in downtown Toronto. <br><br>
To reduce competition with other African restaurants, the company would like to open their Ghanaian restaurants in neighbourhoods that have no or very few African restaurants. The purpose of this project would be to identify such neighbourhoods in downtown Toronto.<br><br>
The target audience of this project is mainly the senior executives the African restaurant chain. The report would enable them to make evidence-based decision on the best neighborhoods to open their restaurants.
</p>

# Data <a name="data"></a>

<p style="font-size:24px">This project will make use of data from:<br><ul style="list-style-type:circle;">
    <li style="font-size:24px"><a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"><b>Toronto Wikipedia page</b></a>: I will scrape postal codes, boroughs and neighbourhoods names from this page. Since the focus of my project is downtown Toronto, I will only consider neighbourhoods in that borough.</li><br>
    <li style="font-size:24px"><b>Geopy package:</b> This package will be used to generate geolocation data (latitude and longitude) for the downtown Toronto neighbourhoods.</li><br>
    <li style="font-size:24px"><b>Foursquare API:</b> From the location data obtained from web scraping and geocoding, I plan to use the Foursquare API to generate African restaurant venue details for the respective the neighbourhoods.</li>

### Below is an example of how feature available from scrapping the Toronto Wikipedia page

In [3]:
# Dropping rows with 'Not assigned' Borough
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True) 
df.reset_index(drop=True)

#Replace missing Neighborhood values with their respective Borough
df['Neighborhood'].fillna(df['Borough'], inplace=True)
df

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


## Below is an example of how the data looks like after adding the geolocation info

In [4]:
import pgeocode

list_lat = []
list_long = []

for index,row in df.iterrows():
    Postal_code = row['Postal code']
    nomi =  pgeocode.Nominatim('ca')
    results = nomi.query_postal_code(Postal_code)
    lat = results['latitude']
    long = results['longitude']
    
    list_lat.append(lat)
    list_long.append(long)
    
df['Latitude'] = list_lat   

df['Longitude'] = list_long

df

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.6518,-79.5076
165,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
168,M7Y,East Toronto,Business reply mail Processing CentrE,43.7804,-79.2505
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.6325,-79.4939
